In [26]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque

In [40]:
class TicTacToe:
    def __init__(self):
        self.reset()

    def reset(self):
        self.board = np.zeros(9, dtype=int)  # 0 = empty, 1 = agent, -1 = opponent
        self.done = False
        self.winner = None
        return self.get_state()

    def get_state(self):
        return self.board.copy()

    def available_actions(self):
        return [i for i in range(9) if self.board[i] == 0]

    def check_winner(self):
        combos = [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]
        for a,b,c in combos:
            s = self.board[a] + self.board[b] + self.board[c]
            if s == 3:
                return 1  # Agent wins
            elif s == -3:
                return -1  # Opponent wins
        if 0 not in self.board:
            return 0  # Draw
        return None  # Game continues

    def step(self, action):
        if self.board[action] != 0 or self.done:
            return self.get_state(), -10, True  # Invalid move penalty
        self.board[action] = 1  # Agent move
        winner = self.check_winner()
        if winner is not None:
            self.done = True
            return self.get_state(), 1 if winner == 1 else 0, True
        return self.get_state(), 0, False

    def step_opp(self):
        # Opponent move (random)
        opp_actions = self.available_actions()
        if opp_actions:
            opp_action = random.choice(opp_actions)
            self.board[opp_action] = -1
        winner = self.check_winner()
        if winner is not None:
            self.done = True
            return self.get_state(), -1 if winner == -1 else 0, True
        return self.get_state(), 0, False


# --- DQN Model ---
class DQN(nn.Module):
    def __init__(self):
        super(DQN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(9, 128),
            nn.ReLU(),
            nn.Linear(128, 9)
        )

    def forward(self, x):
        return self.net(x)

In [41]:
class ReplayBuffer:
    def __init__(self, capacity=10000):
        self.buffer = deque(maxlen=capacity)

    def push(self, transition):
        self.buffer.append(transition)

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        state, action, reward, next_state, done = map(np.array, zip(*batch))
        return (
            torch.tensor(state, dtype=torch.float32),
            torch.tensor(action),
            torch.tensor(reward, dtype=torch.float32),
            torch.tensor(next_state, dtype=torch.float32),
            torch.tensor(done, dtype=torch.float32),
        )

    def __len__(self):
        return len(self.buffer)

In [53]:
# --- Training ---
def train(episodes = 1000):
    env = TicTacToe()
    model = DQN()
    target_model = DQN()
    target_model.load_state_dict(model.state_dict())

    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    buffer = ReplayBuffer()
    batch_size = 64
    gamma = 0.99
    epsilon = 1.0
    epsilon_min = 0.1
    epsilon_decay = 0.999
    update_target_every = 20

    for episode in range(episodes):
        state = env.reset()
        total_reward = 0
        done = False

        while not done:
            if random.random() < epsilon:
                action = random.choice(env.available_actions())
            else:
                with torch.no_grad():
                    q_values = model(torch.tensor(state, dtype=torch.float32))
                    mask = torch.tensor([float('-inf')] * 9)
                    for a in env.available_actions():
                        mask[a] = 0
                    action = torch.argmax(q_values + mask).item()

            next_state, reward, done = env.step(action)
            buffer.push((state, action, reward, next_state, done))
            state = next_state
            total_reward += reward
            if not done:
                _, _, done = env.step_opp()

            if len(buffer) >= batch_size:
                s, a, r, s_, d = buffer.sample(batch_size)
                q_values = model(s).gather(1, a.unsqueeze(1)).squeeze()
                with torch.no_grad():
                    q_next = target_model(s_).max(1)[0]
                    q_target = r + gamma * q_next * (1 - d)
                loss = nn.MSELoss()(q_values, q_target)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        if epsilon > epsilon_min:
            epsilon *= epsilon_decay

        if episode % update_target_every == 0:
            target_model.load_state_dict(model.state_dict())

        if episode % 100 == 0:
            print(f"Episode {episode}, Total Reward: {total_reward}, Epsilon: {epsilon:.3f}")

    torch.save(model.state_dict(), "dqn_tictactoe.pth")
    print("Training complete.")


if __name__ == "__main__":
    train(episodes=0)

Training complete.


In [56]:
def evaluate(model_path="dqn_tictactoe.pth", games=100):
    env = TicTacToe()
    model = DQN()
    model.load_state_dict(torch.load(model_path))
    model.eval()

    wins, losses, draws = 0, 0, 0

    for _ in range(games):
        state = env.reset()
        done = False

        while not done:
            # Agent move
            with torch.no_grad():
                q_values = model(torch.tensor(state, dtype=torch.float32))
                mask = torch.tensor([float('-inf')] * 9)
                for a in env.available_actions():
                    mask[a] = 0
                action = torch.argmax(q_values + mask).item()

            next_state, reward, done = env.step(action)
            if not done:
                next_state, reward, done = env.step_opp()
            state = next_state

        # Final result
        if reward == 1:
            wins += 1
        elif reward == -1:
            losses += 1
        else:
            draws += 1

    print(f"Results over {games} games:")
    print(f"Wins:   {wins}")
    print(f"Losses: {losses}")
    print(f"Draws:  {draws}")


if __name__ == "__main__":
    # Uncomment to run evaluation
    # train()
    evaluate(games=100)


Results over 100 games:
Wins:   70
Losses: 27
Draws:  3


In [57]:
def print_board(state):
    symbols = {1: 'X', -1: 'O', 0: ' '}
    board = [symbols[val] for val in state]
    print("\nBoard:")
    for i in range(3):
        row = " | ".join(board[i * 3:(i + 1) * 3])
        print(row)
        if i < 2:
            print("---------")
    print()

In [60]:
def play_vs_model(model_path="dqn_tictactoe.pth", user_starts=True):
    env = TicTacToe()
    model = DQN()
    model.load_state_dict(torch.load(model_path))
    model.eval()

    state = env.reset()
    done = False

    print("Welcome to Tic-Tac-Toe!")
    print("You are O (opponent), model is X (agent).")
    print("Board positions are 0-8:")
    print("0 | 1 | 2\n3 | 4 | 5\n6 | 7 | 8\n")

    if not user_starts:
        print("Model goes first.")
        with torch.no_grad():
            q_values = model(torch.tensor(state, dtype=torch.float32))
            mask = torch.tensor([float('-inf')] * 9)
            for a in env.available_actions():
                mask[a] = 0
            action = torch.argmax(q_values + mask).item()
        state, _, done = env.step(action)
        print_board(state)

    while not done:
        # --- User move ---
        user_action = -1
        valid = env.available_actions()
        while user_action not in valid:
            try:
                user_action = int(input(f"Your move (available: {valid}): "))
            except ValueError:
                continue

        # Apply user move
        env.board[user_action] = -1
        result = env.check_winner()
        if result is not None:
            done = True
            print_board(env.get_state())
            if result == -1:
                print("You win!")
            elif result == 0:
                print("It's a draw!")
            else:
                print("Model wins!")
            return

        # --- Model move ---
        state = env.get_state()
        with torch.no_grad():
            q_values = model(torch.tensor(state, dtype=torch.float32))
            mask = torch.tensor([float('-inf')] * 9)
            for a in env.available_actions():
                mask[a] = 0
            action = torch.argmax(q_values + mask).item()

        state, _, done = env.step(action)
        print_board(state)

        if done:
            result = env.check_winner()
            if result == 1:
                print("Model wins!")
            elif result == -1:
                print("You win!")
            else:
                print("It's a draw!")


if __name__ == "__main__":
    # train()
    # evaluate()
    play_vs_model(user_starts=True)

Welcome to Tic-Tac-Toe!
You are O (opponent), model is X (agent).
Board positions are 0-8:
0 | 1 | 2
3 | 4 | 5
6 | 7 | 8

Your move (available: [0, 1, 2, 3, 4, 5, 6, 7, 8]): 0

Board:
O |   | X
---------
  |   |  
---------
  |   |  

Your move (available: [1, 3, 4, 5, 6, 7, 8]): 4

Board:
O |   | X
---------
  | O |  
---------
  |   | X

Your move (available: [1, 3, 5, 6, 7]): 5

Board:
O |   | X
---------
  | O | O
---------
  | X | X

Your move (available: [1, 3, 6]): 6

Board:
O |   | X
---------
X | O | O
---------
O | X | X

Your move (available: [1]): 1

Board:
O | O | X
---------
X | O | O
---------
O | X | X

It's a draw!
